<a href="https://colab.research.google.com/github/lox228-lox/-/blob/main/Sirius_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyod

In [12]:
import pandas as pd
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
from pyod.models.lof import LOF
import numpy as np

In [13]:
# Выбор столбцов и преобразование данных
df = pd.read_csv('/content/train.csv')
df.fillna(0, inplace=True)
all_columns = df.columns.tolist()
X = df[all_columns].to_numpy()

In [14]:
# beta
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = list(set(df.columns) - set(numeric_cols))

In [16]:
# beta
for col in categorical_cols:
    df[all_columns] = df[all_columns].astype('category')
    df[all_columns] = df[all_columns].cat.codes

AttributeError: 'DataFrame' object has no attribute 'cat'

In [15]:
# Z-оценка
z_scores = zscore(X)
outliers_z = df[np.abs(z_scores) > 3].index.tolist()

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# IQR
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
outliers_iqr = df[(df < (q1 - 1.5 * iqr)) | (df > (q3 + 1.5 * iqr))].index.tolist()

In [ ]:
# IsolutionForest
clf = IsolationForest(contamination=0.05)
clf.fit(X)
anomaly_scores_if = clf.decision_function(X)
outliers_if = df[anomaly_scores_if < -0.5].index.tolist()  # примерный порог

In [ ]:
# LOF
lof = LOF()
lof.fit(X)
anomaly_scores_lof = lof.score_samples(X)
outliers_lof = df[anomaly_scores_lof > 1].index.tolist()  # примерный порог